In [31]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
def set_folder_path(input_folder):
    if 11<= input_folder <=17:
        input_folder_path = rf"../dataset/raw_data/lettuce/images/training/{input_folder}"
        output_folder_path = rf"../dataset/prep_data/lettuce/images/training/{input_folder}"
    elif 18<= input_folder <=20:
        input_folder_path = rf"../dataset/raw_data/lettuce/images/validation/{input_folder}"
        output_folder_path = rf"../dataset/prep_data/lettuce/images/validation/{input_folder}"
    else:
        input_folder_path = rf"../dataset/practice/before"
        output_folder_path = rf"../dataset/practice/after"

    if input_folder == 11:
        vegetableTotalDays = 37
    elif input_folder == 12:
        vegetableTotalDays = 58
    elif input_folder == 13:
        vegetableTotalDays = 57
    elif input_folder == 14:
        vegetableTotalDays = 54
    elif input_folder == 15:
        vegetableTotalDays = 56
    elif input_folder == 16:
        vegetableTotalDays = 58
    elif input_folder == 17:
        vegetableTotalDays = 56
    elif input_folder == 18:
        vegetableTotalDays = 58
    elif input_folder == 19:
        vegetableTotalDays = 58
    elif input_folder == 20:
        vegetableTotalDays = 56
    else:
        vegetableTotalDays = 0

    return input_folder_path, output_folder_path, vegetableTotalDays

In [33]:
def crop_image(img, left, right, top, bottom):
    return img[top:-bottom, left:-right]

def get_crop_coordinates(width, height):
    if width == 3280 and height == 2464:
        return 440, 440, 32, 32
    elif width == 3000 and height == 4000:
        return 300, 300, 800, 800
    elif width == 4000 and height == 3000:
        return 800, 800, 300, 300
    else:
        raise ValueError(f"Unsupported image size: {width} x {height}.")

def prep_images(input_file, ratio=0.33):
    img = cv2.imread(input_file)

    if img is None:
        print(f"Error: Unable to read the image at {input_file}")
        return

    width, height = img.shape[1], img.shape[0]

    left_margin, right_margin, top_margin, bottom_margin = get_crop_coordinates(width, height)
    cropped_image = crop_image(img, left_margin, right_margin, top_margin, bottom_margin)

    desired_width = 600
    desired_height = 600
    cropped_image = cv2.resize(cropped_image, (desired_width, desired_height))

    img_hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
    green_Lower=np.array([35,50,50])
    green_Upper=np.array([100,255,255])
    red_Lower=np.array([0,50,50])
    red_Upper=np.array([35,255,255])
    img_mask_green=cv2.inRange(img_hsv,green_Lower,green_Upper)
    img_mask_red=cv2.inRange(img_hsv,red_Lower,red_Upper)
    contours_green, _ = cv2.findContours(img_mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_red, _ = cv2.findContours(img_mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask_image = np.zeros_like(cropped_image)
    cv2.drawContours(mask_image, contours_green, -1, (255, 255, 255), thickness=cv2.FILLED)
    cv2.drawContours(mask_image, contours_red, -1, (255, 255, 255), thickness=cv2.FILLED)

    # cropped_image_hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
    # threshold_value = 50
    # _, saturation_mask = cv2.threshold(cropped_image_hsv[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    # mask = np.zeros_like(cropped_image)
    # mask[saturation_mask > 0] = [255, 255, 255]
    # mask_image = cv2.bitwise_and(cropped_image, mask)

    return mask_image

In [34]:
def save_images(input_folder):

    input_folder_path, output_folder_path, vegetableTotalDays = set_folder_path(input_folder)
    input_files = [f for f in os.listdir(input_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for i, input_file in enumerate(input_files, start=1):
        mask_image = prep_images(os.path.join(input_folder_path, input_file))

        output_file = f"{input_folder}_{vegetableTotalDays}_{i}.png"
        output_folder_path = rf"../dataset/practice/after"
        output_file_path = os.path.join(output_folder_path, output_file)

        gray_mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(output_file_path, gray_mask_image)

        # hsv_mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2HSV)
        # cv2.imwrite(output_file_path, hsv_mask_image)


In [35]:
for input_folder in range(11, 21):
    print(input_folder)
    save_images(input_folder)

# save_images(24)

11
12
13
14
15
16
17
18
19
20
